In [26]:
import numpy as  np
import pandas as pd
import matplotlib
from sklearn import svm
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [2]:
#reading Dataset
sales= pd.read_csv(r"C:\Users\user\Downloads\joined_prdct_sales.csv",  sep = ',',encoding = "ISO-8859-1", header= 0)

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sales
sales.drop(['TotalPrice','ModifyDate','SalesDate','TransactionNumber'],axis=1,inplace=True)

In [4]:
#Sanity Check
sales.head()
sales.shape

(6758125, 13)

In [5]:
sales.describe()

,ProductID,CategoryID,VitalityDays,SalesID,SalesPersonID,CustomerID,Quantity,Discount
count,6.758125e+06,6.758125e+06,2.661575e+06,6.758125e+06,6.758125e+06,6.758125e+06,6.758125e+06,1.351194e+06
mean,2.265474e+02,5.864278e+00,6.610745e+01,3.379063e+06,1.199972e+01,4.939567e+04,1.300401e+01,1.498871e-01
std,1.304744e+02,3.267920e+00,3.490174e+01,1.950903e+06,6.632689e+00,2.850504e+04,7.209701e+00,4.999989e-02
min,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-01
25%,1.140000e+02,3.000000e+00,3.300000e+01,1.689532e+06,6.000000e+00,2.470700e+04,7.000000e+00,1.000000e-01
50%,2.270000e+02,6.000000e+00,7.200000e+01,3.379063e+06,1.200000e+01,4.941400e+04,1.300000e+01,1.000000e-01
75%,3.400000e+02,9.000000e+00,9.500000e+01,5.068594e+06,1.800000e+01,7.407500e+04,1.900000e+01,2.000000e-01
max,4.520000e+02,1.100000e+01,1.200000e+02,6.758125e+06,2.300000e+01,9.875900e+04,2.500000e+01,2.000000e-01


In [6]:
sales.describe()

,ProductID,CategoryID,VitalityDays,SalesID,SalesPersonID,CustomerID,Quantity,Discount
count,6.758125e+06,6.758125e+06,2.661575e+06,6.758125e+06,6.758125e+06,6.758125e+06,6.758125e+06,1.351194e+06
mean,2.265474e+02,5.864278e+00,6.610745e+01,3.379063e+06,1.199972e+01,4.939567e+04,1.300401e+01,1.498871e-01
std,1.304744e+02,3.267920e+00,3.490174e+01,1.950903e+06,6.632689e+00,2.850504e+04,7.209701e+00,4.999989e-02
min,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-01
25%,1.140000e+02,3.000000e+00,3.300000e+01,1.689532e+06,6.000000e+00,2.470700e+04,7.000000e+00,1.000000e-01
50%,2.270000e+02,6.000000e+00,7.200000e+01,3.379063e+06,1.200000e+01,4.941400e+04,1.300000e+01,1.000000e-01
75%,3.400000e+02,9.000000e+00,9.500000e+01,5.068594e+06,1.800000e+01,7.407500e+04,1.900000e+01,2.000000e-01
max,4.520000e+02,1.100000e+01,1.200000e+02,6.758125e+06,2.300000e+01,9.875900e+04,2.500000e+01,2.000000e-01


In [7]:
#count the missing Values
sales.isnull().values.sum()

13542590

In [8]:
#Calculate the percentage of Missing Values
sales.isnull().sum()*100/sales.shape[0]

ProductID         0.000000
ProductName       0.000000
Price             0.000000
CategoryID        0.000000
Class             0.000000
Resistant        30.983890
IsAllergic       28.782821
VitalityDays     60.616665
SalesID           0.000000
SalesPersonID     0.000000
CustomerID        0.000000
Quantity          0.000000
Discount         80.006378
dtype: float64

In [9]:
#dropping the na cells
sales_clean = sales.dropna()

In [10]:
#Sanity check
sales_clean.shape
sales_clean.isnull().sum()

ProductID        0
ProductName      0
Price            0
CategoryID       0
Class            0
Resistant        0
IsAllergic       0
VitalityDays     0
SalesID          0
SalesPersonID    0
CustomerID       0
Quantity         0
Discount         0
dtype: int64

In [11]:
sales_clean["ProductName"] = sales_clean["ProductName"].astype('category')
sales_clean.dtypes

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


ProductID           int64
ProductName      category
Price              object
CategoryID          int64
Class              object
Resistant          object
IsAllergic         object
VitalityDays      float64
SalesID             int64
SalesPersonID       int64
CustomerID          int64
Quantity            int64
Discount          float64
dtype: object

In [12]:
sales_clean.head(15)

,ProductID,ProductName,Price,CategoryID,Class,Resistant,IsAllergic,VitalityDays,SalesID,SalesPersonID,CustomerID,Quantity,Discount
29576,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,888,8,78056,20,0.1
29580,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,1674,11,28156,8,0.1
29596,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,8072,2,43444,11,0.1
29597,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,8114,8,57986,15,0.2
29603,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,12987,7,36591,10,0.1
29605,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,13332,4,67525,18,0.1
29610,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,15217,3,85564,22,0.2
29614,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,16235,7,76673,20,0.2
29615,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,16569,12,50200,13,0.2
29626,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,22531,7,55397,15,0.1


In [13]:
#converting ProductName variable into numerical datatype as the algorithm cannot perform analysis on object/string datatype
sales_clean["ProductName_labled"] = sales_clean["ProductName"].cat.codes

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
sales_clean

,ProductID,ProductName,Price,CategoryID,Class,Resistant,IsAllergic,VitalityDays,SalesID,SalesPersonID,CustomerID,Quantity,Discount,ProductName_labled
29576,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,888,8,78056,20,0.1,44
29580,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,1674,11,28156,8,0.1,44
29596,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,8072,2,43444,11,0.1,44
29597,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,8114,8,57986,15,0.2,44
29603,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,12987,7,36591,10,0.1,44
29605,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,13332,4,67525,18,0.1,44
29610,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,15217,3,85564,22,0.2,44
29614,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,16235,7,76673,20,0.2,44
29615,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,16569,12,50200,13,0.2,44
29626,3,Onions - Cippolini,"9,1379",9,Medium,Weak,False,111.0,22531,7,55397,15,0.1,44


In [15]:
sales_clean["Class"] = sales_clean["Class"].astype('category')
sales_clean.dtypes
sales_clean["Class_labled"] = sales_clean["Class"].cat.codes

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
sales_clean.dtypes

ProductID                int64
ProductName           category
Price                   object
CategoryID               int64
Class                 category
Resistant               object
IsAllergic              object
VitalityDays           float64
SalesID                  int64
SalesPersonID            int64
CustomerID               int64
Quantity                 int64
Discount               float64
ProductName_labled        int8
Class_labled              int8
dtype: object

In [17]:
sales_clean.drop(['ProductName','Price','Resistant','Class','Resistant','IsAllergic','VitalityDays','SalesID','SalesPersonID','Quantity','Discount'],axis=1,inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [18]:
sales_clean.drop('CustomerID',axis=1,inplace=True)
sales_clean.dtypes

ProductID             int64
CategoryID            int64
ProductName_labled     int8
Class_labled           int8
dtype: object

In [19]:
X = sales_clean.drop('CategoryID',axis=1)  
y = sales_clean['CategoryID']

In [20]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [27]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 200,
                               max_depth=5, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=200,
            splitter='best')

In [28]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 200,
max_depth=11, min_samples_leaf=10)
clf_entropy.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=11,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=200,
            splitter='best')

In [29]:
y_pred = clf_gini.predict(X_test)
y_pred

array([ 3,  5,  3, ..., 10,  3,  9], dtype=int64)

In [30]:
y_pred_en = clf_entropy.predict(X_test)
y_pred_en

array([10,  5, 10, ...,  1,  1,  9], dtype=int64)

In [31]:
print("Accuracy is",accuracy_score(y_test,y_pred)*100)

Accuracy is 58.51453230757375


In [32]:
print("Accuracy is",accuracy_score(y_test,y_pred_en)*100)

Accuracy is 100.0
